In [ ]:
import pandas as pd
import numpy as np
import os 
import re

#1. CARICAMENTO DATASET
file_path = os.path.join('datasets' , 'steam-games.csv')
df = pd.read_csv(file_path)

# 2. PULIZIA PREZZO (Deve essere fatta PRIMA del drop)
def clean_price(price_val):
    if pd.isna(price_val):
        return 0.0
    price_str = str(price_val).lower().strip()
    if 'free' in price_str or 'demo' in price_str or 'play' in price_str:
        return 0.0
    clean_s = re.sub(r'[^\d\.]', '', price_str)
    try:
        return float(clean_s)
    except ValueError:
        return 0.0

print("Creazione colonna price_clean...")
# Creiamo la nuova colonna partendo da quella originale
df['price_clean'] = df['discounted_price'].apply(clean_price)

# 3. GESTIONE RIGHE MANCANTI (Drop delle 57 righe senza data)
print(f"Rimozione di {df['release_date'].isnull().sum()} righe senza release_date...")
df.dropna(subset=['release_date'], inplace=True)

# 4. IMPUTAZIONE TESTUALE E RATING
print("Imputazione valori nulli residui...")
text_features = ['genres', 'categories', 'developer', 'publisher', 'about_description']
for col in text_features:
    df[col] = df[col].fillna('')

df['overall_review_%'] = df['overall_review_%'].fillna(0.0)
df['overall_review_count'] = df['overall_review_count'].fillna(0)

# 5. DROP DELLE COLONNE SPORCHE (Ora possiamo farlo in sicurezza)
cols_to_drop = [
    'content_descriptor', 
    'original_price', 
    'discount_percentage', 
    'recent_review', 
    'recent_review_%', 
    'recent_review_count',
    'discounted_price',  # Eliminata perché abbiamo price_clean
    'overall_review'     # Eliminata perché abbiamo overall_review_%
]
df.drop(columns=cols_to_drop, inplace=True, errors='ignore')

# 6. VERIFICA FINALE
print("\n--- STATO FINALE DATASET ---")
print(f"Righe totali: {len(df)}")
print(f"Valori nulli rimasti:\n{df.isnull().sum()}")

# Salvataggio per il Notebook 3
df.to_csv('./datasets/steam-games-clean.csv', index=False)

Creazione colonna price_clean...
Rimozione di 57 righe senza release_date...
Imputazione valori nulli residui...

--- STATO FINALE DATASET ---
Righe totali: 42440
Valori nulli rimasti:
app_id                  0
title                   0
release_date            0
genres                  0
categories              0
developer               0
publisher               0
dlc_available           0
age_rating              0
about_description       0
win_support             0
mac_support             0
linux_support           0
awards                  0
overall_review_%        0
overall_review_count    0
price_clean             0
dtype: int64
